In [1]:
import pandas as pd
import urllib, urllib.request
import fitz
import io
import requests

In [2]:
import re
def strip(s):
    s=' '.join(s.split())
    s=re.sub(r'[^A-Za-z0-9 ]+', '', s)
    return s.lower()

In [3]:
def annotate(i, src):
    date=i["published"]
    title=i["title"]
    authors=i["authors"]
    abstract=strip(i['summary'])
    url = i["links"][-1]["href"]
    tag=i["tags"]
    request = requests.get(url)
    filestream = io.BytesIO(request.content)
    pdf = fitz.open(stream=filestream, filetype="pdf")
    text = ""
    for page in pdf:
        text += page.get_text()
    return [date,title,tag, authors,abstract,strip(text),i["id"], src]
    

In [20]:
urls=[]
urls.append("https://api.semanticscholar.org/graph/v1/paper/search?query=google+books&openAccessPdf&fields=title,publicationDate,authors,abstract,s2FieldsOfStudy,openAccessPdf&limit=100")
urls.append("https://api.semanticscholar.org/graph/v1/paper/search?query=google+books+ngram&openAccessPdf&fields=title,publicationDate,authors,abstract,s2FieldsOfStudy,openAccessPdf&limit=100")
urls.append("https://api.semanticscholar.org/graph/v1/paper/search?query=google+ngram&openAccessPdf&fields=title,publicationDate,authors,abstract,s2FieldsOfStudy,openAccessPdf&limit=100")
urls.append("https://api.semanticscholar.org/graph/v1/paper/search?query=google+n gram&openAccessPdf&fields=title,publicationDate,authors,abstract,s2FieldsOfStudy,openAccessPdf&limit=100")
urls.append("https://api.semanticscholar.org/graph/v1/paper/search?query=google+books+n gram&openAccessPdf&fields=title,publicationDate,authors,abstract,s2FieldsOfStudy,openAccessPdf&limit=100")

urls.append("https://api.semanticscholar.org/graph/v1/paper/search?query=hathitrust&openAccessPdf&fields=title,publicationDate,authors,abstract,s2FieldsOfStudy,openAccessPdf&limit=100")

urls.append("https://api.semanticscholar.org/graph/v1/paper/search?query=internet+archive&openAccessPdf&fields=title,publicationDate,authors,abstract,s2FieldsOfStudy,openAccessPdf&limit=100")
urls.append("https://api.semanticscholar.org/graph/v1/paper/search?query=gutenberg&openAccessPdf&fields=title,publicationDate,authors,abstract,s2FieldsOfStudy,openAccessPdf&limit=100")


In [54]:
urls

['https://api.semanticscholar.org/graph/v1/paper/search?query=internet+archive&openAccessPdf&fields=title,publicationDate,authors,abstract,s2FieldsOfStudy,openAccessPdf&limit=100&offset=6900',
 'https://api.semanticscholar.org/graph/v1/paper/search?query=google+books&openAccessPdf&fields=title,publicationDate,authors,abstract,s2FieldsOfStudy,openAccessPdf&limit=100&offset=6800',
 'https://api.semanticscholar.org/graph/v1/paper/search?query=google+ngram&openAccessPdf&fields=title,publicationDate,authors,abstract,s2FieldsOfStudy,openAccessPdf&limit=100&offset=6900',
 'https://api.semanticscholar.org/graph/v1/paper/search?query=google+n gram&openAccessPdf&fields=title,publicationDate,authors,abstract,s2FieldsOfStudy,openAccessPdf&limit=100&offset=6800']

In [21]:
ret=[]

In [76]:
import time

In [77]:
while urls:
    url=urls.pop(0)
    response = requests.get(url)
    try:
        print(len(response.json()["data"]))
    except:
        urls.append(url)
        print(url)
        time.sleep(1)
        continue
    if "next" in response.json():
        url=url.split("&offset=")[0]
        urls.append(url+"&offset="+str(response.json()["next"]))
    for i in response.json()["data"]:
        title=i["title"]
        date=i["publicationDate"]
        try:
            abstract=i["abstract"]
        except:
            abstract=""
        url=i["openAccessPdf"]["url"]
        tags=[]
        for t in i["s2FieldsOfStudy"]:
            tags.append(t["category"])
        authors=[]
        for a in i["authors"]:
            authors.append(a["name"])
        ret.append([date,title,tags, authors,abstract, url])

100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
https://api.semanticscholar.org/graph/v1/paper/search?query=google+ngram&openAccessPdf&fields=title,publicationDate,authors,abstract,s2FieldsOfStudy,openAccessPdf&limit=100&offset=9700
100
100
100
100
100
100
100
100
100
https://api.semanticscholar.org/graph/v1/paper/search?query=internet+archive&openAccessPdf&fields=title,publicationDate,authors,abstract,s2FieldsOfStudy,openAccessPdf&limit=100&offset=9900
100
https://api.semanticscholar.org/graph/v1/paper/search?query=google+ngram&openAccessPdf&fields=title,publicationDate,authors,abstract,s2FieldsOfStudy,openAccessPdf&limit=100&offset=9900
https://api.semanticscholar.org/graph/v1/paper/search?query=google+n gram&openAccessPdf&fields=title,publicationDate,authors,abstract,s2FieldsOfStudy,openAccessPdf&limit=100&offset=9900
https://api.semanticscholar.org/graph/v1/paper/search?query=internet+archive&openAccessPdf&fields=title,publicationDate,authors,abs

KeyboardInterrupt: 

In [78]:
full=pd.DataFrame(ret)

In [79]:
full

,0,1,2,3,4,5
0,2022-06-27,Culturomic Explorations of Literary Prominence...,[Art],"[J. Tyrkkö, I. Mäkinen]",The availability of databases of digitised lit...,https://www.journals.vu.lt/knygotyra/article/d...
1,2022-11-22,The Possibilities and Limitations of Using Goo...,[Business],"[D. Madsen, Kåre Slåtten]",Google Books Ngram Viewer (GNV) is an analytic...,https://www.mdpi.com/2075-4698/12/6/171/pdf?ve...
2,2021-10-24,Citation analysis of Ph.D. theses with data fr...,"[Computer Science, Psychology]",[P. Donner],None,https://link.springer.com/content/pdf/10.1007/...
3,2021-07-30,Corpus Linguistics Representations on Age Grou...,"[Psychology, Psychology]",[B. Silva],The goal of the research \nlays on identifying...,http://www.scirp.org/journal/PaperDownload.asp...
4,2020-03-25,"Google Books, Scopus, Microsoft Academic, and ...","[Computer Science, Political Science, Education]","[K. Kousha, M. Thelwall]",A research doctorate normally culminates in pu...,https://direct.mit.edu/qss/article-pdf/1/2/479...
...,...,...,...,...,...,...
43373,2016-02-26,GENERAL MECHANISMS OF CITATION SYSTEM OF SCIEN...,[Computer Science],"[Віктор Дмитрович Гогунський, Володимир Олекса...",Аналіз життєвого циклу наукових публікацій сві...,http://pm.khpi.edu.ua/article/download/2413-30...
43374,2020-10-24,Utilization Learning Management System (LMS) o...,[Computer Science],"[M. A. H. Putra, Mutiani Mutiani, Jumriani Jum...",Penerapan teknologi di bidang pendidikan yang ...,https://ppjp.ulm.ac.id/journals/index.php/kss/...
43375,2020-08-28,The Development of an Online Platform for Stud...,"[Political Science, Education]","[S. Sharov, O. Filatova, T. Sharova]",The authors of this article adduce facts accor...,http://www.temjournal.com/content/93/TEMJourna...
43376,2013-12-31,Comparative Literature in Chinese: A Survey of...,"[History, Economics]",[Miaomiao Wang],"Part of the American Studies Commons, Comparat...",https://docs.lib.purdue.edu/cgi/viewcontent.cg...


In [80]:
full[1]=full[1].apply(strip)
full[4]=full[4].fillna("")
full[4]=full[4].apply(strip)

In [81]:
keep=[]
for i,row in full.iterrows():
    match=False
    s=row[1]+" "+row[4]
    matches=["internet archive", "hathi", "gutenberg"]
    for m in matches:
        if "google" in s:
            if "book" in s:
                match=True
            if "ngram" in s:
                match=True
        if m in s:
            match=True
    keep.append(match)

In [82]:
full["keep"]=keep

In [83]:
full=full[full["keep"]]

In [85]:
full.columns=["date", "title", "tag", "author", "abstract", "url", "keep"]

In [87]:
full=full.drop_duplicates(["title"])

In [88]:
full

,date,title,tag,author,abstract,url,keep
0,2022-06-27,culturomic explorations of literary prominence...,[Art],"[J. Tyrkkö, I. Mäkinen]",the availability of databases of digitised lit...,https://www.journals.vu.lt/knygotyra/article/d...,True
1,2022-11-22,the possibilities and limitations of using goo...,[Business],"[D. Madsen, Kåre Slåtten]",google books ngram viewer gnv is an analytical...,https://www.mdpi.com/2075-4698/12/6/171/pdf?ve...,True
2,2021-10-24,citation analysis of phd theses with data from...,"[Computer Science, Psychology]",[P. Donner],,https://link.springer.com/content/pdf/10.1007/...,True
3,2021-07-30,corpus linguistics representations on age grou...,"[Psychology, Psychology]",[B. Silva],the goal of the research lays on identifying s...,http://www.scirp.org/journal/PaperDownload.asp...,True
4,2020-03-25,google books scopus microsoft academic and men...,"[Computer Science, Political Science, Education]","[K. Kousha, M. Thelwall]",a research doctorate normally culminates in pu...,https://direct.mit.edu/qss/article-pdf/1/2/479...,True
...,...,...,...,...,...,...,...
42822,2017-01-04,keeping it real from faces and features to soc...,"[Computer Science, Computer Science]",[Anja Bechmann],this paper wants to supplement computational t...,http://scholarspace.manoa.hawaii.edu/bitstream...,True
42948,2022-11-17,pelatihan elearning via google classroom untuk...,[],"[Rizqi Putri Nourma Budiarti, Afib Rulyansah, ...",tim pengabdian kepada masyarakat prodi pgsd un...,https://ukinstitute.org/journals/ib/article/do...,True
43099,2016-12-14,ujian online mahasiswa ilmu komputer berbasis ...,"[Computer Science, Education]",[L. Abdillah],information technology influence higher educat...,http://arxiv.org/pdf/1701.06428,True
43325,2021-01-13,identifikasi praktik dan hambatan guru dalam a...,[Education],"[Hajra Yansa, Heri Retnawati]",this study aims to identify teachers practices...,http://e-journal.hamzanwadi.ac.id/index.php/je...,True


In [89]:
full.to_csv("semantic_meta.csv")

In [93]:
texts=[]
for i,row in full.iterrows():
    if i%20==0:
        print(i)
    l=row["url"]
    t=row["title"][:20]
    try:
        request = requests.get(l)
        filestream = io.BytesIO(request.content)
        pdf = fitz.open(stream=filestream, filetype="pdf")
    except:
        texts.append("")
        continue
    text = ""
    for page in pdf:
        text += page.get_text()
    #texts.append(strip(text))
    f = open("semantic/"+t+".txt", "a", encoding="utf-8")
    f.write(strip(text))
    f.close()

0
20
40
60
80
120
140
240
260
320
340
360
460
480
500
520
540
560
580
700
720
740
760
780
800
820
840
860
980
1260
1280
1300
1320
1340
1380
1400
1440
1460
1480
1500
1520
1540
1560
1580
1600
1620
1900
1980
2000
2020
2060
2120
2140
2160
2220
2260
2300
2580
2600
2620
2680
2700
2720
2740
2760
2840
2860
3200
3220
3260
3280
3320
3340
3360
3440
3460
3780
3800
3820
3840
3860
3880
3900
3920
3980
4000
4380
4420
4440
4600
4620
4660
4700
4980
5020
5040
5060
5160
5220
5240
5580
5600
5640
5660
5720
5820
5860
6180
6200
6240
6320
6380
6400
6440
6460
6540
6560
6800
6820
6840
6860
6920
6960
6980
7040
7100
7380
7400
7420
7440
7460
7480
7560
7620
7980
8000
8020
8040
8060
8080
8260
8600
8620
8640
8780
8800
8820
9080
9100
9120
9140
9160
9180
9280
9320
9340
9820
10200
10220
10240
10400
10700
10720
11200
11220
11240
11260
11380
11420
11580
11600
11620
11640
11660
16540
20940
23000
23760
27440
28460
28640
28840
29700
29760
30280
30620
30840
31680
31920
32680
40080


In [104]:
texts=[]
for t in full["title"]:
    try:
        with open("semantic/"+t[:20]+".txt", "r", encoding="utf-8") as f:
            text = f.read()
    except:
        text=""
    texts.append(text)

In [105]:
full["text"]=texts

C:\Users\lucia\AppData\Local\Temp\ipykernel_12036\1131976992.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full["text"]=texts


In [99]:
t[:20]

'a free virtual reali'

In [101]:
with open("semantic/"+t[:20]+".txt", "r", encoding="utf-8") as f:
    text = f.read().rstrip()

In [106]:
full["text"].value_counts()

In [108]:
full=full[full["text"]!=""]

In [109]:
full.to_csv("semantic.csv")